Given a wikipedia page for a specific league season (e.g. 'https://en.wikipedia.org/wiki/1967%E2%80%9368_League_of_Ireland') finds the final league table on the page and returns a dataframe.
It presupposes that the table is the first table in the page with 'Pos' as the first header. Works ok for all examples I've tried.

In [1]:
import requests
import urllib.request
import urllib.error
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen


In [2]:
def league_table_scraper(url):
  
  while True:
    try:
       req = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e:
        return []
    except urllib.error.URLError as e:
        return []
    else:
      #print('URL Exists')
      html = urlopen(url) 
      soup = BeautifulSoup(html, 'html.parser')

      tables = soup.find_all('table')
      for i,table in enumerate(tables):
        rows=table.find_all('tr')
        heads = rows[0].find_all('th')
        if (len(heads)>0):
          h0 = heads[0].text
          if(h0.find('Pos')>=0):
            league_table_no = i
            break

      league_table = tables[league_table_no]
      table_headers = league_table.find_all('th')
      col_heads = np.array([h.text for h in table_headers])
      idx_Pos = np.where(col_heads=='Pos\n')[0][0]
      idx_team = idx_Pos+1
      idx_Pld = np.where(col_heads=='Pld\n')[0][0]
      idx_W = np.where(col_heads=='W\n')[0][0]
      idx_L = np.where(col_heads=='L\n')[0][0]
      idx_D = np.where(col_heads=='D\n')[0][0]
      idx_GF = np.where(col_heads=='GF\n')[0][0]
      idx_GA = np.where(col_heads=='GA\n')[0][0]
      idx_Pts = np.where(col_heads=='Pts\n')[0][0]

      rows = league_table.find_all('tr')
      team = [];played = [];won = [];lost = [];drew = [];gf = [];ga = [];pts = [];pos=[];gd=[]
      for r in rows[1:]:
        els = r.find_all(['td','th'])
        if len(els)>=8:
          team.append(els[idx_team].text.replace('\n',''))
          pos.append(int(els[idx_Pos].text.replace('\n','')))
          played.append(int(els[idx_Pld].text.replace('\n','')))
          won.append(int(els[idx_W].text.replace('\n','')))
          lost.append(int(els[idx_L].text.replace('\n','')))
          drew.append(int(els[idx_D].text.replace('\n','')))
          gf.append(int(els[idx_GF].text.replace('\n','')))
          ga.append(int(els[idx_GA].text.replace('\n','')))
          pts.append(els[idx_Pts].text.replace('\n',''))

      # not all tables have a goal difference columns so I add it here
      table = pd.DataFrame({'Pos':pos,'Team':team,'Pld':played,'W':won,'D':drew,'L':lost,'GF':gf,'GA':ga,'Pts':pts})
      table['GD'] =  table.GF.astype(int) - table.GA.astype(int)
      # rearrange the columns
      cols = list(table.columns.values);cols = cols[:-2]+['GD']+[cols[-2]];table = table[cols]
      table.set_index('Pos',inplace=True)

      return table


In [3]:
url='https://en.wikipedia.org/wiki/1967%E2%80%9368_League_of_Ireland'
#url='https://en.wikipedia.org/wiki/1963-64_Bundesliga'
#url='https://en.wikipedia.org/wiki/1995-96_Bundesliga'
#url='https://en.wikipedia.org/wiki/1936-37_La_Liga'
table = league_table_scraper(url)
print(table)

                    Team  Pld   W  D   L  GF  GA  GD Pts
Pos                                                     
1              Waterford   22  16  2   4  59  18  41  34
2                Dundalk   22  14  2   6  44  24  20  30
3            Cork Celtic   22  12  6   4  40  27  13  30
4        Shamrock Rovers   22  11  5   6  44  26  18  27
5               Drogheda   22  10  6   6  33  29   4  26
6               Limerick   22   9  2  11  35  45 -10  20
7             Shelbourne   22   8  3  11  32  36  -4  19
8             Drumcondra   22   6  7   9  31  35  -4  19
9    St Patrick Athletic   22   7  4  11  29  46 -17  18
10       Cork Hibernians   22   6  5  11  19  28  -9  17
11          Sligo Rovers   22   6  4  12  24  48 -24  16
12             Bohemians   22   2  4  16  20  48 -28   8
